<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/803/w2v2_finetune_on_asr_best_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch torchaudio
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

In [3]:
from datasets import load_dataset
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, TrainingArguments, Trainer

# Load the dataset
dataset = load_dataset("HamdanXI/uclass_asr_v3")

def prepare_dataset(batch):
    # Process audio
    speech_array = batch['audio']['array']
    speech_tensor = torch.tensor(speech_array)

    # Pad to the maximum length
    # max_length = 205 * 16000  # 205 seconds at 16kHz
    # padded_speech = torch.nn.functional.pad(input=speech_tensor, pad=(0, max_length - speech_tensor.shape[0]) if speech_tensor.shape[0] < max_length else (0, 0), mode='constant', value=0.0)

    # Tokenize the labels
    with processor.as_target_processor():
        labels = processor(batch['transcript']).input_ids

    return {"input_values": speech_array, "labels": labels}

# Load tokenizer and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Apply the mapping function
dataset = dataset.map(prepare_dataset, remove_columns=["audio"])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    group_by_length=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=128,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    save_total_limit=2,
)

import torch
from torch.nn.utils.rnn import pad_sequence

class DataCollatorForWav2Vec2:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        # Ensure each input value is a tensor, then pad
        input_values = [torch.tensor(feature['input_values'], dtype=torch.float) for feature in features]
        labels = [torch.tensor(feature['labels'], dtype=torch.long) for feature in features]

        # Pad input values and labels
        input_values_padded = pad_sequence(input_values, batch_first=True, padding_value=0.0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=self.processor.tokenizer.pad_token_id)

        return {
            'input_values': input_values_padded,
            'labels': labels_padded
        }

data_collator = DataCollatorForWav2Vec2(processor=processor)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=data_collator,
    tokenizer=processor
)

# Testing the trainer on a small subset to ensure it processes data correctly
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.71 GiB. GPU 0 has a total capacity of 22.17 GiB of which 836.88 MiB is free. Process 47284 has 21.34 GiB memory in use. Of the allocated memory 17.98 GiB is allocated by PyTorch, and 3.14 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)